In [1]:
#import keras

#import keras.backend as K
#from keras.models import load_model

import kipoi

import numpy as np
import scipy

import pandas as pd


In [2]:

model = kipoi.get_model('DeepSEA/predict')


Using downloaded and verified file: /home/ubuntu/.kipoi/models/DeepSEA/predict/downloaded/model_files/weights/89e640bf6bdbe1ff165f484d9796efc7


In [3]:

model.model


Sequential(
  (0): ReCodeAlphabet()
  (1): ConcatenateRC()
  (2): Sequential(
    (0): Conv2d(4, 320, kernel_size=(1, 8), stride=(1, 1))
    (1): Threshold(threshold=0, value=1e-06)
    (2): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.2, inplace=False)
    (4): Conv2d(320, 480, kernel_size=(1, 8), stride=(1, 1))
    (5): Threshold(threshold=0, value=1e-06)
    (6): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.2, inplace=False)
    (8): Conv2d(480, 960, kernel_size=(1, 8), stride=(1, 1))
    (9): Threshold(threshold=0, value=1e-06)
    (10): Dropout(p=0.5, inplace=False)
    (11): Lambda()
    (12): Sequential(
      (0): Lambda()
      (1): Linear(in_features=50880, out_features=925, bias=True)
    )
    (13): Threshold(threshold=0, value=1e-06)
    (14): Sequential(
      (0): Lambda()
      (1): Linear(in_features=925, out_features=919, bias=True)
    )
    (15):

In [11]:
simple_torch_model = model.model[2]

In [21]:
simple_torch_model

Sequential(
  (0): Conv2d(4, 320, kernel_size=(1, 8), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
  (3): Dropout(p=0.2, inplace=False)
  (4): Conv2d(320, 480, kernel_size=(1, 8), stride=(1, 1))
  (5): ReLU()
  (6): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
  (7): Dropout(p=0.2, inplace=False)
  (8): Conv2d(480, 960, kernel_size=(1, 8), stride=(1, 1))
  (9): ReLU()
  (10): Dropout(p=0.5, inplace=False)
  (11): Lambda()
  (12): Sequential(
    (0): Lambda()
    (1): Linear(in_features=50880, out_features=925, bias=True)
  )
  (13): ReLU()
  (14): Sequential(
    (0): Lambda()
    (1): Linear(in_features=925, out_features=919, bias=True)
  )
  (15): Sigmoid()
)

In [20]:
simple_torch_model[1] = torch.nn.ReLU()
simple_torch_model[5] = torch.nn.ReLU()
simple_torch_model[9] = torch.nn.ReLU()
simple_torch_model[13] = torch.nn.ReLU()

In [22]:
import torch
from torch.autograd import Variable

input_np = np.random.uniform(0, 1, (1, 4, 1, 1000))
input_var = Variable(torch.FloatTensor(input_np))


In [23]:
from pytorch2keras import pytorch_to_keras

k_model = pytorch_to_keras(simple_torch_model, input_var, [(4, 1, 1000,)], verbose=True) 


INFO:pytorch2keras:Converter is called.
DEBUG:pytorch2keras:Input_names:
DEBUG:pytorch2keras:['input_0']
DEBUG:pytorch2keras:Output_names:
DEBUG:pytorch2keras:['output_0']


graph(%input_0 : Float(1, 4, 1, 1000),
      %0.weight : Float(320, 4, 1, 8),
      %0.bias : Float(320),
      %4.weight : Float(480, 320, 1, 8),
      %4.bias : Float(480),
      %8.weight : Float(960, 480, 1, 8),
      %8.bias : Float(960),
      %12.1.weight : Float(925, 50880),
      %12.1.bias : Float(925),
      %14.1.weight : Float(919, 925),
      %14.1.bias : Float(919)):
  %11 : Float(1, 320, 1, 993) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[1, 8], pads=[0, 0, 0, 0], strides=[1, 1]](%input_0, %0.weight, %0.bias), scope: Sequential/Conv2d[0] # /home/ubuntu/anaconda3/envs/pytorch_p36_fresh/lib/python3.6/site-packages/torch/nn/modules/conv.py:342:0
  %12 : Float(1, 320, 1, 993) = onnx::Relu(%11), scope: Sequential/ReLU[1] # /home/ubuntu/anaconda3/envs/pytorch_p36_fresh/lib/python3.6/site-packages/torch/nn/functional.py:914:0
  %13 : Float(1, 320, 1, 248) = onnx::MaxPool[kernel_shape=[1, 4], pads=[0, 0, 0, 0], strides=[1, 4]](%12), scope: Sequential/Dropout[3] # /hom

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:[(4, 1, 1000)]
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input_0.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output_0.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight 0.bias with shape (320,).
DEBUG:onnx2keras:Found weight 0.weight with shape (320, 4, 1, 8).
DEBUG:onnx2keras:Found weight 12.1.bias with shape (925,).
DEBUG:onnx2keras:Found weight 12.1.weight with shape (925, 50880).
DEBUG:onnx2keras:Found weight 14.1.bias with shape (919,).
DEBUG:onnx2keras:Found weight 14.1.weight with shape (919, 925).
DEBUG:onnx2keras:Found weight 4.bias with shape (480,).
DEBUG:onnx2keras:Found weight 4.weight with shape (480, 320, 1, 8).
DEBUG:onnx2keras:Found weight 8.bias with shape (960,).
DEBUG:onnx2keras:Found weight 8.weight with shape (960, 480, 1, 8).
DEBUG:onnx2keras:Found input input_0 with shape (4, 1, 1000)
DEBUG:onnx2keras:######
DEB

DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 22).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:unsqueeze:Work with numpy types.
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Concat
DEBUG:onnx2keras:node_name: 25
DEBUG:onnx2keras:node_params: {'axis': 0, 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 23).
DEBUG:onnx2keras:Check input 1 (name 24).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:concat:Concat numpy arrays.
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Reshape
DEBUG:onnx2keras:node_name: 26
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2k

In [24]:
k_model

In [25]:

k_model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_0 (InputLayer)         [(None, 4, 1, 1000)]      0         
_________________________________________________________________
11 (Conv2D)                  (None, 320, 1, 993)       10560     
_________________________________________________________________
12 (Activation)              (None, 320, 1, 993)       0         
_________________________________________________________________
13_pad (ZeroPadding2D)       (None, 320, 1, 993)       0         
_________________________________________________________________
13 (MaxPooling2D)            (None, 320, 1, 248)       0         
_________________________________________________________________
14 (Conv2D)                  (None, 480, 1, 241)       1229280   
_________________________________________________________________
15 (Activation)              (None, 480, 1, 241)       0     

In [29]:

k_model.get_layer('11').data_format


'channels_first'

In [26]:
#Save keras-version of DeepSEA

k_model.save("deepsea_keras.h5")
